In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

from lib import utils,built

In [3]:
args = pickle.load(open('arguments.pickle', 'rb'))

# Load reference

In [4]:
seq_ref_list = pickle.load(open(os.path.join(args.preprocess_dir, 'seq_ref_list.pickle'), 'rb'))
seq_ref_length = len(seq_ref_list)
seq_ref_length

29903

# หาตำแหน่งแต่ละ alignment ที่ใช้

In [6]:
startstop_df = pd.read_csv(os.path.join(args.preprocess_dir, 'startstop_df.csv'), index_col='id')
snps_df = pd.read_csv(os.path.join(args.preprocess_dir, 'snps_df.csv'))

#เรียง id
id_list = sorted(list(set(startstop_df.index)))
len(id_list)

3406

In [7]:
#จำนวน sample ที่่ไม่่มี snps
len(set(id_list)-(set(list(snps_df['id']))))

17

In [8]:
#สร้าง array ขนาด sample x position จากนั้นใส่่ 1 ในตำแหน่่งที่มี
align_array = np.zeros((len(id_list), seq_ref_length), dtype=str)
snps_max_df = snps_df.copy()

In [9]:
sum_length,snps_max_df,align_array,choose_pos_list = utils.length_max_snps2(align_array,id_list,startstop_df,snps_max_df)

In [10]:
choose_pos_df = pd.DataFrame(choose_pos_list,columns=['id','start_new','end_new','start_end'])
choose_pos_df.head()

,id,start_new,end_new,start_end
0,406973,1,29845,1_29845
1,407987,2,29845,2_29845
2,407988,3,29845,3_29845
3,410535,266,29674,266_29674
4,410536,3,29845,3_29845


# หาตำแหน่ง insertion  และ จำนวน

In [11]:
snps_insertion_df = pd.read_csv(os.path.join(args.preprocess_dir, 'snps_insertion_df.csv'))

In [12]:
snps_insertion_df = snps_insertion_df[(snps_insertion_df['query']=='-')]

In [13]:
snps_insertion_df.head()

,id,query,position,sbjct,start_end,length
748,1081930,-,28251,C,10_29801,29792
749,1081930,-,28253,G,10_29801,29792
3252,1315627,-,28251,C,63_29676,29614
3253,1315627,-,28253,G,63_29676,29614
3266,1315628,-,28251,C,28123_29766,1644


In [14]:
id_insertion = list(set(snps_insertion_df['id']))
len(id_insertion)

11

In [15]:
insertion_list = built.find_insertion_position(id_insertion,snps_insertion_df)

In [16]:
insertion_df = pd.DataFrame(insertion_list, columns=['id','position_ref','n_insert_base','start_end'])
insertion_df = insertion_df.sort_values('id')
insertion_df.head()

,id,position_ref,n_insert_base,start_end
3,1081930,28250,1,10_29801
4,1081930,28251,1,10_29801
5,1315627,28250,1,63_29676
6,1315627,28251,1,63_29676
7,1315628,28250,3,28123_29766


# นำมารวมกัน
- ตำแหน่งที่มี insertion กับ ตำแหน่งในสายที่ใช้สร้าง align array

In [17]:
#เอามารวมกันแล้ว จำนวนอาจลดลง เพราะบาง indertion อยู่ในสายที่เราไม่ได้ใช้ในการสร้าง align array
insertion_sum_df = pd.merge(insertion_df, choose_pos_df, how="right", on=["id", "start_end"])

In [18]:
insertion_sum_df = insertion_sum_df[insertion_sum_df['position_ref'].notna()]

In [19]:
insertion_sum_df = insertion_sum_df[(insertion_sum_df['position_ref']>=insertion_sum_df['start_new']) & (insertion_sum_df['position_ref']<=insertion_sum_df['end_new'])]

In [20]:
insertion_sum_df.head()

,id,position_ref,n_insert_base,start_end,start_new,end_new
2449,1081930,28250.0,1.0,10_29801,10,29801
2450,1081930,28251.0,1.0,10_29801,10,29801
2546,1315627,28250.0,1.0,63_29676,63,29676
2547,1315627,28251.0,1.0,63_29676,63,29676
2549,1315628,28250.0,3.0,28123_29766,28123,29766


In [21]:
insertion_sum_df.to_csv(os.path.join(args.preprocess_dir, 'insertion_df.csv'), index=False)

In [22]:
insertion_sum_df

,id,position_ref,n_insert_base,start_end,start_new,end_new
2449,1081930,28250.0,1.0,10_29801,10,29801
2450,1081930,28251.0,1.0,10_29801,10,29801
2546,1315627,28250.0,1.0,63_29676,63,29676
2547,1315627,28251.0,1.0,63_29676,63,29676
2549,1315628,28250.0,3.0,28123_29766,28123,29766
2662,1519355,28262.0,4.0,89_29740,89,29740
2750,1543939,28262.0,4.0,45_29792,45,29792
2830,1620144,28262.0,4.0,50_29759,50,29759
3201,1914659,28262.0,4.0,18_29787,18,29787
3203,1914660,28262.0,4.0,22260_29793,22260,29793
